# Data Exploration for tunning a foundation model 

Project evnironment set up:
- Since wer are running the code locally, there is a few libraries and steps that we need to complete to get ready

In [1]:
# Import needed library 
from utils import authenticate
from google.cloud import bigquery
from google.oauth2 import service_account
import vertexai 
import dotenv

In [2]:
# provide credential to to access google cloud platform
PROJECT_ID = "de-tunning-foundation-model"
credentials = service_account.Credentials.from_service_account_file('credentials.json')
REGION = "us-central1"

Now we need to initialize Vertex AI services

In [3]:
vertexai.init(project = PROJECT_ID,
              location = REGION,
              credentials = credentials)

Next, we will import big query to use as our data warehouse. First we will need to initialize the client to start interacting with the data warehouse, next, send SQL and retrieve data into the notebook
- This dataset include questions, answer, and metadata related to stack overflow questions. With this dataset, there are tables with data


In [4]:
bq_client = bigquery.Client(project=PROJECT_ID,
                            credentials = credentials)

In [5]:
QUERY_TABLES = """
SELECT
  table_name
FROM
  `bigquery-public-data.stackoverflow.INFORMATION_SCHEMA.TABLES`
"""

In [6]:
query_job = bq_client.query(QUERY_TABLES)

This will print out all the tables in the stackoverflow directory

In [7]:
for row in query_job:
    for value in row.values():
        print(value)

posts_answers
users
posts_orphaned_tag_wiki
posts_tag_wiki
stackoverflow_posts
posts_questions
comments
posts_tag_wiki_excerpt
posts_wiki_placeholder
posts_privilege_wiki
post_history
badges
post_links
tags
votes
posts_moderator_nomination


# Data Retrieval

- Next we will fetch some data from the data warehouse and store it in pandas df for visualization

In [8]:
# select 3 questions from the post_questions data warehouse 
INSPECT_QUERY = """
SELECT * 
FROM `bigquery-public-data.stackoverflow.posts_questions`
LIMIT 3
"""

Store the dataset in a data frame

In [9]:
import pandas as pd

In [10]:
# execute the query
query_job = bq_client.query(INSPECT_QUERY)

Take the results of the query --> create an arrow table (which is part of Apache Framework) --> which goes into a Pandas dataframe.
This allows for data to be in a format which is easier to read and explore with Pandas.

Apache Arrow is an open-source project that provides a standard for in-memory data representation. It's designed to enable efficient data processing and communication between systems. An Arrow table is essentially a collection of Arrow arrays organized into columns. These arrays are stored in a columnar memory format, which makes operations on large datasets much faster compared to traditional row-based storage. This is especially beneficial in analytics and data processing tasks.

In [11]:
stack_overflow_df = query_job.result()\
    .to_arrow()\
        .to_pandas()
        
stack_overflow_df.head()

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,73210679,az acr login raises DOCKER_COMMAND_ERROR with ...,<p>Windows 11 with wsl2 ubuntu-22.04.</p>\n<p>...,73247188.0,1,0,NaT,2022-08-02 16:16:31.810000+00:00,NaN,2022-08-05 09:00:14.693000+00:00,2022-08-02 16:32:13.700000+00:00,None,11226740,None,11226740,None,1,0,azure-container-registry|docker-daemon,256
1,73284406,Run Azure log query from the command line with...,<p>I am trying to get the Azure log query data...,NaN,2,0,NaT,2022-08-08 21:53:16.703000+00:00,NaN,2022-08-10 04:49:20.527000+00:00,2022-08-09 08:12:13.920000+00:00,None,2395282,None,19123691,None,1,0,azure|azure-devops|azure-application-insights|...,256
2,73250763,Error CS0246: The type or namespace name 'Stre...,<p>I have these errors when trying to write th...,73251390.0,1,0,NaT,2022-08-05 13:43:25.850000+00:00,NaN,2022-08-05 16:40:00.610000+00:00,2022-08-05 16:40:00.610000+00:00,None,17654458,None,17654458,None,1,3,c#|unity3d,512


# Dealing with large datasets

In the real world, large datasets for LLMs often don't fit into memory. 

We can see this right below 

In [12]:
QUERY_ALL = """
select 
* 
from `bigquery-public-data.stackoverflow.posts_questions` q""" 

In [13]:
query_job = bq_client.query(QUERY_ALL)

In [14]:
try: 
    stack_overflow_df = query_job.result()\
        .to_arrow()\
            .to_pandas()    
except Exception as e:
    print('The dataframe is too large to load into memeory.', e)

The dataframe is too large to load into memeory. 403 Response too large to return. Consider specifying a destination table in your job configuration. For more details, see https://cloud.google.com/bigquery/troubleshooting-errors

Location: US
Job ID: 1c575932-ec23-49f9-bb5d-5ea6414d5a99



Apparently the query is too large to be fit in memory

#### Joining Tables and Query Optimization

- When working with (large) data, query optimizing is needed in order to save time and resources.
- Select questions as `input_text` (column 1), answers as `output_text` (column 2).
- Take the questions from `posts_questions` and answers from `posts_answers`.
- Join the questions and their corresponding accepted answers based on their same `unique ID`.
- Making sure the question is about `Python`, and that it `has an answer`. And the date the question was posted is on or after `2020-01-01`
- Limit as 10,000

In [15]:
QUERY = """ 
select 
concat(q.title, q.body) as input_text,
a.body as output_text
from `bigquery-public-data.stackoverflow.posts_questions` q
join `bigquery-public-data.stackoverflow.posts_answers` a
on q.accepted_answer_id = a.id
where 
q.accepted_answer_id is not null
and q.tags like '%python%'
and a.creation_date > '2020-01-01'
limit 10000

"""

In [16]:
query_job = bq_client.query(QUERY)

In [17]:
# convert the result to data frame 
stack_overflow_df = query_job.result().to_arrow().to_pandas()

stack_overflow_df.head()

,input_text,output_text
0,igraph: get dict with nodes and corresponding ...,<p>You can recover the original networkx node ...
1,"The first coordinates never work, but the foll...",<p>Those first four lines:</p>\n<pre><code>b =...
2,Is there a performance deficit not splitting e...,<p>Depends if your <code>function_one</code>/<...
3,Comparing Multiple Columns to Multiple lists a...,<p>I don't fully understand which output you w...
4,unable to change date format while loading exc...,<p>Using pd.to_datetime() allows for the use o...


# Adding instruction for LLMs

### Adding Instructions

- Instructions for LLMs have been shown to improve
model performance and generalization to unseen tasks [(Google, 2022)](https://arxiv.org/pdf/2210.11416.pdf).
- Wihtout the instruction, it is only question and answer. Model might not understand what to do.
- With the instructions, the model gets a guideline as to what task to perform.

In [18]:
INSTRUCTION_TEMPLATE = f"""\
Please answer the following Stackoverflow question on Python. \
Answer it like you are a developer answering Stackoverflow questions.

Stackoverflow question:
"""

With the new instruction, we will embed this instruction to the front of all the questions

In [19]:
stack_overflow_df['input_text_instruction'] = INSTRUCTION_TEMPLATE + ' ' + stack_overflow_df['input_text']

# Prepare dataset for tunning

- Use the spliting ratio 80/20

In [20]:
from sklearn.model_selection import train_test_split

In [21]:
train, eval = train_test_split(
    stack_overflow_df, 
    test_size = 0.2,
    random_state = 6701
)

# Different datasets and flow 
- it's important to version data because it allows reproducibility, traceability, maintainability of machine learning models. 
- it's important to get the time stamp for the dataset 

In [22]:
import datetime

In [39]:
date = datetime.datetime.now().strftime("%H:%d:%m:%Y")
date = date.replace(':', '_')

- Generate a `jsonl` file
- Then name it as `tune_data_stack_overflow_python_qa-{date}`

In [41]:
# for train dataset
cols = ['input_text_instruction','output_text']
tune_jsonl = train[cols].to_json(orient="records", lines=True)

In [45]:
training_data_filename = f"tune_data_stack_overflow_\
                            python_qa_{date}.jsonl"

In [46]:
with open(training_data_filename, "w") as f:
    f.write(tune_jsonl)

In [49]:
# for test dataste 
test_jsonl = eval[cols].to_json(orient="records", lines=True)

In [47]:
eval_data_filename = f"eval_data_stack_overflow_\
                            python_qa_{date}.jsonl" 

In [50]:
with open(eval_data_filename, "w") as f: 
    f.write(test_jsonl)
    